<a href="https://colab.research.google.com/github/Gustavo-2212/llm-gsi073-2025/blob/main/atv02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets

In [2]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

X = X[y != 1]
y = y[y != 1]
y = torch.tensor(y, dtype=torch.float32)
y[y == 0] = -1

X = torch.tensor(X, dtype=torch.float32)

In [3]:
n_features = X.shape[1]
w = torch.randn(n_features, 1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

learning_rate = 0.01
epochs = 300
optimizer = optim.Adam([w, b], lr=learning_rate)

In [4]:
for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = X @ w + b
    perda_de_classificacao = torch.clamp(1 - y.view(-1, 1) * y_pred, min=0).mean()
    perda_de_distancia_entre_classes = 0.5 * torch.sum(w ** 2)

    loss = perda_de_distancia_entre_classes + perda_de_classificacao

    loss.backward()
    optimizer.step()

    if(epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss={loss.item():.4f}")

Epoch 100/300, Loss=0.1653
Epoch 200/300, Loss=0.1426
Epoch 300/300, Loss=0.1250


In [7]:
with torch.no_grad():
    scores = X @ w + b
    predictions = torch.where(scores > 0, torch.tensor(2.0), torch.tensor(-1.0))
    correct = (predictions.view(-1) == y).float().sum()
    accuracy = correct / y.shape[0]

    print(f"Acurácia: {accuracy.item() * 100:.2f}%")

Acurácia: 100.00%


## Comparação: SVM x Regressão Logística

> Embora ambos sejam classificadores lineares e frequentemente alcancem acurácias similares em problemas simples, eles funcionam de formas fundamentalmente diferetes:

1. Regressão Logística
* Objetivo probabilístico, tentando maximizar a verossimilhança dos dados;
* Função de perda (Log Loss - BCEWithLogitsLoss), penalizando o erro de probabilidade suavemente;
* Foco considerando todos ospontos;
* Saída é um valor de probabilidade (0 a 1).

2. SVM
* Objetivo geométrico tentando maximizar a marge (a linha mais larga possível) entre as classes;
* Função de perda Hinger Loss (1 - y*pred) zera o erro se o ponto estiver correto e fora da margem;
* Foco apenas nos vetores de suporte (pontos difíceis perto da fronteira). Pontos fáceis são ignorados;
* Distância para o hiperplano (score real) é retornado na saída.

> Como as classes (Setosa e Virgínica) são linearmente separáveis com facilidade, ambos os modelos provavelmente chegarão a 100% de acurácia, mas a reta desenhada pelo SVM será a que está mais "centralizada" entre os dois grupos.

### Usando nn.Linear


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data
y = iris.target

X = X[y != 1]
y = y[y != 1]
y = torch.tensor(y, dtype=torch.float32)
y[y == 0] = -1

X = torch.tensor(X, dtype=torch.float32)
n_features = X.shape[1]

modelo = nn.Linear(n_features, 1)

learning_rate = 0.01
epochs = 300
optimizer = optim.Adam(modelo.parameters(), lr=learning_rate)

for epoch in range(epochs):
    optimizer.zero_grad()

    y_pred = modelo(X)

    perda_de_classificacao = torch.clamp(1 - y.view(-1, 1) * y_pred, min=0).mean()
    perda_de_distancia_entre_classes = 0.5 * torch.sum(modelo.weight ** 2)

    loss = perda_de_distancia_entre_classes + perda_de_classificacao

    loss.backward()
    optimizer.step()

    if(epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss={loss.item():.4f}")

with torch.no_grad():
    scores = modelo(X)
    preds = torch.where(scores > 0, torch.tensor(2.0), torch.tensor(-1.0))
    acc = (preds.view(-1) == y).float().mean()
    print(f"Acurácia Final: {acc.item():.2%}")

Epoch 100/300, Loss=0.1311
Epoch 200/300, Loss=0.1077
Epoch 300/300, Loss=0.0989
Acurácia Final: 100.00%
